In [82]:
import sys, os
densepose_path = os.path.abspath('./detectron2/projects/DensePose')
#if densepose_path not in sys.path:
#    sys.path.insert(0, densepose_path)

#from detectron2.projects.DensePose import densepose
from modules.data.utils import read_video_frames

In [83]:
from modules.cloth_masker import AutoMasker

In [84]:
resume_path = "zhengchong/CatVTON"
dataset_name = 'dresscode'
data_root_path = '/mnt/d/dresscode/upper_body'
image_path = '000000_0.jpg'

In [85]:
automasker = AutoMasker(
    densepose_ckpt=os.path.join(resume_path, "DensePose"),
    schp_ckpt=os.path.join(resume_path, "SCHP"),
    device='cuda', 
)

In [75]:
from PIL import Image, ImageFilter

In [95]:
image = Image.open(os.path.join(data_root_path, 'images', image_path))

In [96]:
conditions = automasker(image, mask_type='upper')

In [99]:
conditions['densepose'].save('000004_5.png')
conditions['mask'].save('000004_4.png')
#conditions['schp_atr']

In [89]:
densepose_gray = automasker.densepose_processor(image, resize=512, colormap=None)
densepose_gray.save('000004_5_uv.png')

In [93]:
from detectron2.data.detection_utils import read_image
dp = automasker.densepose_processor
img = read_image(os.path.join(data_root_path, 'images', image_path), format="BGR")
with torch.no_grad():
    outputs = dp.predictor(img)["instances"]

In [70]:
from densepose.vis.extractor import DensePoseResultExtractor, DensePoseOutputsExtractor
if isinstance(outputs.pred_densepose, DensePoseChartPredictorOutput):
    extractor = DensePoseResultExtractor()
elif isinstance(outputs.pred_densepose, DensePoseEmbeddingPredictorOutput):
    extractor = DensePoseOutputsExtractor()

dat = extractor(outputs)[0]
uv = dat[0].uv
uv = uv.detach().cpu().numpy()
print(uv.shape)  # (H, W, 2)
print(uv.min(), uv.max()) # U/V ranges in 0–1

(2, 885, 253)
-0.016818535 1.0358227


In [56]:
import numpy as np
import torch
densepose = np.load(os.path.join(data_root_path, 'dense', '000000_5_uv.npz'))
print(densepose['uv'].shape)  # (H, W, 2)
print(densepose['uv'].min(), densepose['uv'].max()) # U/V ranges in 0–1

(2, 1024, 768)
0.0 1.0046434


In [51]:
import torch
from detectron2.data.detection_utils import read_image
from densepose.structures import DensePoseChartPredictorOutput, DensePoseEmbeddingPredictorOutput
from densepose.vis.extractor import DensePoseResultExtractor, DensePoseOutputsExtractor
dp = DensePose(model_path=os.path.join(resume_path, 'DensePose'), device="cuda")
# Grab DensePose predictor from AutoMasker
# predictor = dp.predictor
def run_densepose_dump(image_path):
    """
    Run DensePose inference and return results with UV maps.
    Args:
        dp: DensePose instance (from modules.densepose.DensePose)
        image_path: Path to input image
    Returns:
        dict with keys:
            - 'file_name': str
            - 'scores': tensor of shape [N] (if N detections)
            - 'pred_boxes_XYXY': tensor of shape [N, 4]
            - 'pred_densepose': list of N DensePoseChartResult objects
                Each has .uv of shape [2, H_bbox, W_bbox] in local bbox coordinates
    """
    # Load image
    img = read_image(image_path, format="BGR")
    
    # Run inference
    with torch.no_grad():
        outputs = dp.predictor(img)["instances"]
    
    # Build result dictionary
    result = {"file_name": image_path}
    result["scores"] = outputs.get("scores").cpu()
    result["pred_boxes_XYXY"] = outputs.get("pred_boxes").tensor.cpu()
    # Extract DensePose results (list of N instances)
    if isinstance(outputs.pred_densepose, DensePoseChartPredictorOutput):
        extractor = DensePoseResultExtractor()
    elif isinstance(outputs.pred_densepose, DensePoseEmbeddingPredictorOutput):
        extractor = DensePoseOutputsExtractor()
    
    result["pred_densepose"] = extractor(outputs)[0]
    
    return result

In [52]:
data = run_densepose_dump(os.path.join(data_root_path, 'images', image_path))
data

{'file_name': '/mnt/d/dresscode/upper_body/images/000000_0.jpg',
 'scores': tensor([0.9997]),
 'pred_boxes_XYXY': tensor([[225.9432,   6.3674, 479.6125, 891.5373]]),
 'pred_densepose': [DensePoseChartResultWithConfidences(labels=tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'), uv=tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0

In [57]:
uv = data['pred_densepose'][0].uv
uv = uv.detach().cpu().numpy()
print(uv.shape)  # (H, W, 2)
print(uv.min(), uv.max()) # U/V ranges in 0–1
# Save as npz file
#np.savez("uv_data.npz", uv=uv)

(2, 885, 253)
-0.016818542 1.0358227


In [58]:
result['uv'][0].shape

KeyError: 'uv'

In [53]:
uv = result['pred_densepose'][0].uv
uv = uv.detach().cpu().numpy()
print(uv.shape)  # (H, W, 2)
print(uv.min(), uv.max()) # U/V ranges in 0–1
# Save as npz file
#np.savez("uv_data.npz", uv=uv)

(2, 901, 280)
-0.034850176 1.0361613


In [57]:
#https://github.com/facebookresearch/detectron2/issues/1751
import matplotlib.pyplot as plt
i = result['pred_densepose'][0].labels.cpu().numpy()
uv = result['pred_densepose'][0].uv.cpu().numpy()
iuv = np.stack((uv[1,:,:], uv[0,:,:], i * 0,))
plt.imshow(np.transpose(iuv, (1,2,0)))
plt.show()
plt.savefig("output123.png")

iuv = np.stack((uv[1,:,:], uv[0,:,:], i))
iuv = np.transpose(iuv, (1,2,0))
plt.imshow(iuv / 255)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
/tmp/ipykernel_411/949048166.py:7: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [61]:
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import resize

# Original image dimensions
H_img, W_img = 1024, 768
# Extract DensePose labels and UV
labels = result['pred_densepose'][0].labels.cpu().numpy()
uv = result['pred_densepose'][0].uv.cpu().numpy()

# Build IUV stack: (H_roi, W_roi, 3)
iuv = np.stack((uv[1, :, :], uv[0, :, :], labels * 0))  # U=uv[1], V=uv[0]
iuv = np.transpose(iuv, (1, 2, 0))
# Prepare full-size canvas
full_canvas = np.zeros((H_img, W_img, 3), dtype=iuv.dtype)

# Bounding box coordinates (float → int)
x0, y0, x1, y1 = result['pred_boxes_XYXY'][0].cpu().numpy().astype(int)
# --- Ensure ROI matches the bbox slice size ---
h_box = y1 - y0
w_box = x1 - x0
if (iuv.shape[0] != h_box) or (iuv.shape[1] != w_box):
    iuv = resize(iuv, (h_box, w_box), preserve_range=True, anti_aliasing=True)

# Paste into full-size canvas
full_canvas[y0:y1, x0:x1, :] = iuv
# Save directly without Matplotlib figure scaling
full_canvas = np.clip(full_canvas, 0, 1)  # make sure in [0,1]
plt.imsave("output123.png", full_canvas)
full_canvas.shape

(1024, 768, 3)